<div class="alert alert-block alert-success">
    <h1 align="center">Telecom Customer Churn Modelling</h1>
    <h4 align="center" >Alireza Esmaeilpour</h4>
    <h6 align="center"><a href="https://alireza-esp.ir/">Website</a></h6>
    <h6 align="center"><a href="https://github.com/Alireza-Esp">Github</a></h6>
    <h6 align="center"><a href="https://www.kaggle.com/alirezaesmaeilpour">Kaggle</a></h6>
</div>

# 🔵 Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sklearn as sk

import warnings
warnings.filterwarnings('ignore')

# 🔵 Load the data

In [3]:
data = pd.read_csv("https://github.com/Alireza-Esp/Telecom-Customer-Churn-Modelling/raw/refs/heads/main/data/telecom-customer-churn-v2.csv")

In [4]:
data

,City,Latitude,Longitude,Gender,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,...,Streaming Music,Unlimited Data,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Customer Status,Churn Category,Churn Reason
0,Los Angeles,33.964131,-118.272783,Male,No,No,2,Yes,No,DSL,...,No,Yes,0.00,0,20.94,129.09,1,Churned,Competitor,Competitor made better offer
1,Los Angeles,34.059281,-118.307420,Female,No,Yes,2,Yes,No,Fiber optic,...,No,Yes,0.00,0,18.24,169.89,2,Churned,Other,Moved
2,Los Angeles,34.048013,-118.293953,Female,No,Yes,8,Yes,Yes,Fiber optic,...,Yes,Yes,0.00,0,97.20,917.70,3,Churned,Other,Moved
3,Los Angeles,34.062125,-118.315709,Female,Yes,Yes,28,Yes,Yes,Fiber optic,...,Yes,Yes,0.00,0,136.92,3182.97,3,Churned,Other,Moved
4,Los Angeles,34.039224,-118.266293,Male,No,Yes,49,Yes,Yes,Fiber optic,...,Yes,Yes,0.00,0,2172.17,7208.47,1,Churned,Competitor,Competitor had better devices
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,Landers,34.341737,-116.539416,Female,No,No,72,Yes,No,No,...,No,No,19.31,0,1639.44,3039.53,5,Stayed,NaN,NaN
7028,Adelanto,34.667815,-117.536183,Male,Yes,Yes,24,Yes,Yes,DSL,...,Yes,Yes,48.23,0,865.20,2807.47,3,Stayed,NaN,NaN
7029,Amboy,34.559882,-115.637164,Female,Yes,Yes,72,Yes,Yes,Fiber optic,...,Yes,Yes,45.38,0,2135.52,9453.04,4,Stayed,NaN,NaN
7030,Angelus Oaks,34.167800,-116.864330,Female,Yes,Yes,11,No,No phone service,DSL,...,No,Yes,27.24,0,0.00,319.21,4,Stayed,NaN,NaN


# 🔵 Preprocessing